In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from scipy.stats import mode,zscore
from matplotlib import pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_absolute_error,mean_squared_error

data = pd.read_json("../../../../data/data.json")

data= data[data["capture"]== "tw2"]

data.sample(2)

,index,mac,timestamp,rssi,frequency,iq,angle,distance,capture,switch_order,ref,phases,aoa
24165,148,c300000a7a48,2024-11-29 13:58:15.787000+00:00,-59,2480,"[184, -39, 9, 187, -189, -11, 52, -183, 179, 6...",-4,300.0,tw2,2-2-2-0-5-6-14-13-8-10,"[-0.20886533840000002, 1.5227050935, -3.083457...","[2.8681914858, 0.0181798151, -2.2338144766, 1....","[-0.4623574057, 11.8683899216, 10.8267606124, ..."
36953,106,c300000a7a48,2024-11-29 18:09:01.053000+00:00,-45,2426,"[110, 149, -182, 70, -37, -189, 195, -14, -21,...",50,150.0,tw2,2-2-2-0-5-6-14-13-8-10,"[0.9348544693, 2.7744188198, -1.7641185894, -0...","[-2.2132643781, -2.8535372065, 3.1415926536, 2...","[51.4165107027, 65.1736076135, 62.8085131922, ..."


# Coluna

Esse modelo usa como entrada cada coluna IQ que sabemos que existe no pacote, logo precisamos gerá-las

In [3]:
#2-2-2-0-5-6-14-13-8-10 = 8*2 elementos em cada coluna, 2*8 de ref
# CTE = 20*8 => 160-4-8= 148 => 148*2 = 74 =>74//8 => 9 com resto 2, logo precisamos ignorar a última amostra 
data["iqs"] = data["iq"].map(lambda x: [x[i:i+8*2] for i in range(2*8,len(x),8*2)][:-1]) 
data["training_value"]= data["iqs"]

In [4]:
tsamples =[]
tlabels =[]

vsamples=[]
vlabels=[]

# Separamos no dataset para que a gnt seja capaz de depois analizar o conteúdo por pacote
tdataset, vdataset = train_test_split(data,test_size=0.3,random_state=10)

for row in tdataset.iterrows():
    for element in row[1]["training_value"]:
        tsamples.append(element)
        tlabels.append(row[1]["angle"])

for row in vdataset.iterrows():
    for element in row[1]["training_value"]:
        vsamples.append(element)
        vlabels.append(row[1]["angle"])

X_train = tsamples
X_val = vsamples


y_train = tlabels
y_val = vlabels 


In [5]:
tv=pd.DataFrame([{"angulo":angle, "treinamento":y_train.count(angle),"validacao" :y_val.count(angle)} for angle in set(y_val)])
tv.set_index("angulo")

,treinamento,validacao
angulo,,
0,9477,4050
1,1809,900
2,1890,819
3,1989,720
4,1980,729
5,1863,846
6,1836,873
7,1827,882
8,1971,738


In [6]:
model = xgb.XGBRegressor(enable_categorical=True)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [7]:
#Making predictions on the test set
predictions = model.predict(X_val)

#Calculating accuracy
mae_model =mean_absolute_error(y_val, predictions)
mse_model =mean_squared_error(y_val, predictions)

print(f"MAE= Modelo: {mae_model}")
print(f"MSE= Modelo: {mse_model}")

MAE= Modelo: 5.343964484279398
MSE= Modelo: 62.39233910735445


In [8]:
aoa_val=[]
results_labels= []
results_values = []
for row in vdataset.iterrows():
    pred=[]
    for element in row[1]["training_value"]:
        pred.append(model.predict([element])[0])
    results_labels.append((row[1]["angle"]))
    results_values.append(np.mean(pred))
    aoa_val.append(np.mean(row[1]["aoa"]))

In [9]:
#Calculating accuracy

mae_model =mean_absolute_error(results_labels, results_values)
mse_model =mean_squared_error(results_labels, results_values)

mae_aoa =mean_absolute_error(results_labels, aoa_val)
mse_aoa =mean_squared_error(results_labels, aoa_val)

print(f"MAE= Modelo: {mae_model}\t AoA={mae_aoa}")
print(f"MSE= Modelo: {mse_model}\t AoA={mse_aoa}")

MAE= Modelo: 2.687910679951765	 AoA=17.91555958745627
MSE= Modelo: 20.840156883766724	 AoA=789.3761852881698


In [10]:
model.save_model("./regressor_iq_row.json")